In [44]:
import os
import asyncio
import threading
import time
import base64
from datetime import datetime, timedelta
from datetime import time
from typing import List, Dict, Any, Optional
import sqlite3
from contextlib import asynccontextmanager
import json
from fastapi.responses import FileResponse


from fastapi import FastAPI, HTTPException, BackgroundTasks, Depends, File, UploadFile, Form
from fastapi.middleware.cors import CORSMiddleware
from pydantic import BaseModel, Field, validator
from dotenv import load_dotenv
import uvicorn
#from playsound import playsound
import io

# LangGraph and LangChain imports
from langgraph.graph import StateGraph, END, START
from langchain_core.messages import HumanMessage
from langchain_groq import ChatGroq
from fastapi.staticfiles import StaticFiles 
from typing import TypedDict,Annotated,Literal
from langchain.prompts import PromptTemplate

import nest_asyncio
nest_asyncio.apply()

# Load environment variables
load_dotenv()

True

In [4]:
llm = ChatGroq(
            model="openai/gpt-oss-120b",  # Using available model
            temperature=0.1,           
        )

In [1]:
import json


result={
    "summary": "A woman is being pursued and verbally threatened by an unknown person. She repeatedly pleads for the individual to stop, asks for help, threatens to call the police, and expresses fear of being harmed.",
    "threat_level": 9.0,
    "distress_words": [
        "stop",
        "don't touch me",
        "help",
        "someone help me",
        "I will call the police",
        "please don't hurt me",
        "I haven't done anything",
        "please stop following me",
        "leave me alone"
    ]
}

In [2]:
result = json.loads(result)

TypeError: the JSON object must be str, bytes or bytearray, not dict

In [22]:
template1 = """
You are an AI Assistant who is specially designed to analyse the transcripted audio file which is received from the phone of a woman who might get harassed at {real_time}. You have to generate a threat level between 5 to 10 based on the following transcript. If this prompt is running which exclusively means that there is night time and user input already a dangerous place:

Audio Transcript: {TAudio}

Remember:
Provide high rating if you found some distress sentences like "don't touch me", "I will complain about you", "Please don't do this to me"... and many other distress sentences.

Return your response in the exact JSON format:
{{
    "summary": "Provide a proper summary for the event",
    "threat_level": 5.0,
    "distress_words": ["list", "of", "distress", "words"]
}}
"""
prompt = PromptTemplate(
            input_variables=[ 'TAudio','TAudio'],  # Fixed variable names
            template=template1
        )


In [23]:
chain=prompt|llm

In [24]:
result=chain.invoke({ "real_time":"2025-08-19T02:20:00Z",
                     "TAudio":"""{Woman: "Please stop following me, I don't know you"
    Unknown person: "Come on, don't be like that"
    Woman: "Don't touch me! Leave me alone!"
    Unknown person: "Just talk to me for a minute"
    Woman: "Help! Someone help me! Don't do this to me!"
    Woman: "I will call the police if you don't stop"
    Unknown person: "Nobody can hear you here"
    Woman: "Please don't hurt me, I haven't done anything"}"""})

In [ ]:
result.content#It returns a JSON String


'{\n    "summary": "A woman is being pursued and verbally threatened by an unknown individual. She repeatedly pleads for the person to stop, asks for help, threatens to call the police, and expresses fear of being harmed.",\n    "threat_level": 9.0,\n    "distress_words": [\n        "stop",\n        "don\'t touch me",\n        "help",\n        "someone help me",\n        "I will call the police",\n        "please don\'t hurt me",\n        "I haven\'t done anything"\n    ]\n}'

In [27]:
result = json.loads(result.content)

In [28]:
result

{'summary': 'A woman is being pursued and verbally threatened by an unknown individual. She repeatedly pleads for the person to stop, asks for help, threatens to call the police, and expresses fear of being harmed.',
 'threat_level': 9.0,
 'distress_words': ['stop',
  "don't touch me",
  'help',
  'someone help me',
  'I will call the police',
  "please don't hurt me",
  "I haven't done anything"]}

In [29]:
result['summary']

'A woman is being pursued and verbally threatened by an unknown individual. She repeatedly pleads for the person to stop, asks for help, threatens to call the police, and expresses fear of being harmed.'

Return function

In [30]:
class Location(BaseModel):
    latitude:  float = Field(..., ge=-90.0, le=90.0, description="Degrees north or south of the equator")
    longitude: float = Field(..., ge=-180.0, le=180.0, description="Degrees east or west of prime meridian")

    # Optional: custom string pattern instead of float
    @validator("latitude", "longitude", pre=True)
    def parse_str_to_float(cls, v):
        # Allow passing "12.345" or 12.345
        return float(v)


C:\Users\asus\AppData\Local\Temp\ipykernel_4100\1250525760.py:6: PydanticDeprecatedSince20: Pydantic V1 style `@validator` validators are deprecated. You should migrate to Pydantic V2 style `@field_validator` validators, see the migration guide for more details. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  @validator("latitude", "longitude", pre=True)


In [31]:
class Summarization(BaseModel):
    location:Location
    summary:str
    time:datetime
    distress_words:List[str]
    threat_level:float


In [58]:
def experiment():
    result={'summary': 'A woman is being pursued and verbally threatened by an unknown individual. She repeatedly pleads for the person to stop, asks for help, threatens to call the police, and expresses fear of being harmed.',
 'threat_level': 9.0,
 'distress_words': ['stop',
  "don't touch me",
  'help',
  'someone help me',
  'I will call the police',
  "please don't hurt me",
  "I haven't done anything"]}
    raw = {
        'location':{
        "latitude":  "12.9716",
        "longitude": "77.5946"
    },
         'time':  "2025-08-18T19:29:00",
         'summary': result['summary'],
         'threat_level': result['threat_level'],
         'distress_words': result['distress_words']
}
    return Summarization(**raw)
    

In [59]:
a=experiment()

In [47]:
a.location

Location(latitude=12.9716, longitude=77.5946)

In [48]:
a.time

datetime.datetime(2025, 8, 18, 19, 29)

In [37]:
a.distress_words

['stop',
 "don't touch me",
 'help',
 'someone help me',
 'I will call the police',
 "please don't hurt me",
 "I haven't done anything"]

In [33]:
      raw = {
        'location':{
        "latitude":  "12.9716",
        "longitude": "77.5946"
    },
         'time':  "2025-08-18T19:29:00",
         'summary': result['summary'],
         'threat_level': result['threat_level'],
         'distress_words': result['distress_words']
}
    return Summarization(**raw)


IndentationError: unindent does not match any outer indentation level (<string>, line 11)

Time

In [40]:
night_start = time(0, 0)    # 00:00 (12:00 AM)
night_end = time(4, 0)   

In [49]:
night_end

datetime.time(4, 0)

In [41]:
raw={
'real_time':  "2025-08-18T19:29:00"}

In [42]:
raw['real_time']

'2025-08-18T19:29:00'

In [46]:
raw['real_time']time()

SyntaxError: invalid syntax (2174212619.py, line 1)

In [ ]:
if night_start <= raw['real_time'].time() <= night_end:
            template = self.template1

datetime.time(0, 0)

In [ ]:
from datetime import datetime, time

# Your time window
night_start = time(0, 0)    # 00:00 (12:00 AM)
night_end = time(4, 0)      # 04:00 AM

# Your timestamp data
raw = {'real_time': "2025-08-18T19:29:00"}

# 1. Parse the string to datetime
timestamp = datetime.fromisoformat(raw['real_time'])

# 2. Extract just the time component
current_time = timestamp.time()

# 3. Now you can compare properly
if night_start <= current_time <= night_end:
    print("It's nighttime (between 12AM-4AM)")
else:
    print("It's not nighttime")

It's not nighttime


In [55]:
timestamp = datetime.fromisoformat(raw['real_time'])

In [57]:
timestamp.time()

datetime.time(19, 29)

In [54]:

night_end


datetime.time(4, 0)

In [52]:
night_start

datetime.time(0, 0)

In [53]:
current_time

datetime.time(19, 29)